## ECE 196: NBA Data Collection

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import os

In [2]:
wd = os.getcwd()

In [3]:
year1 = 18
year2 = 19

pd.set_option('display.max_columns', 500)

base_url = ('https://stats.nba.com/teams/boxscores/?Season=20{}-{}&SeasonType=Regular%20Season')


### Download Chrome webdriver here: https://sites.google.com/a/chromium.org/chromedriver/downloads
### Unzip the file, keep it in the same folder you downloaded it in
### Change the following line with the path to your chromedriver
driver = webdriver.Chrome(f'{wd}/chromedriver') # change to your path

In [4]:
local_filename = 'NBA_Team_Data_{}-{}'

In [ ]:
driver.get(base_url.format(year1,year2))
print('Got url')

In [15]:
# Finds page selector, and clicks on 'All' (this can be done manually if needed)
page_selector = driver.find_elements_by_tag_name('select')[2]
page_selector.click()
page_selector.find_elements_by_tag_name("option")[0].click()

ElementClickInterceptedException: Message: element click intercepted: Element <select class="stats-table-pagination__select ng-pristine ng-untouched ng-valid ng-not-empty" ng-model="vm.data.page" ng-options="p for p in vm.data.pages" aria-invalid="false">...</select> is not clickable at point (1209, 3). Other element would receive the click: <button type="button" class="style__link_2QXEv">...</button>
  (Session info: chrome=87.0.4280.141)


In [ ]:
# Takes about 30 seconds to run
%time soup = BeautifulSoup(driver.page_source, 'html5lib')

In [ ]:
nba_table = soup.find('div', {'class':'nba-stat-table'})

In [ ]:
type(nba_table)

In [ ]:
team_columns = []
for row in nba_table.find('thead').find_all('th'):
    #print(row.text)
    team_columns.append(row.text)
team_columns

In [ ]:
for row in range(len(team_columns)):
    team_columns[row] = team_columns[row].strip(' ').strip('\n').strip(' ').replace('\xa0', ' ')
team_columns

In [ ]:
%%time 
all_rows = nba_table.find('tbody').find_all('tr')
team_data = []
for row in all_rows:
    row_values = []
    for val in row.find_all('td'):
        row_values.append(val.text.strip(' ').strip('\n').strip(' '))
    team_data.append(row_values)
team_data[:2]

In [ ]:
team_data_df = pd.DataFrame(data=team_data, columns=team_columns)
team_data_df.head()

We drop the +/- column to not give away who wins.

In [ ]:
team_data_df = team_data_df.drop('+/-', axis=1)

In [ ]:
team_data_df.to_csv('NBA_Team_Data_{}-{}.csv'.format(year1, year2), index=False)